In [ ]:
import os
import yaml
import jsonlines
from langchain.schema import Document
def load_config():
    """
    Carga la configuración de la aplicación desde el archivo 'config.yaml'.

    Returns:
        Un diccionario con la configuración de la aplicación.
    """
    root_dir = os.path.dirname(os.path.abspath(__file__))
    with open(os.path.join(root_dir, "config.yaml")) as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

def get_file_path():
    """
    Obtiene la ruta al archivo de base de datos JSONL especificado en la configuración de la aplicación.

    Returns:
        La ruta al archivo de base de datos JSONL.
    """
    config = load_config()

    root_dir = os.path.dirname(os.path.abspath(__file__))
    parent_dir = os.path.join(root_dir, "..")

    return os.path.join(parent_dir, config["jsonl_database_path"])


In [ ]:
from utils import get_openai_api_key 
from langchain.chat_models import ChatOpenAI

get_openai_api_key()

llm = ChatOpenAI(
  model_name="gpt-3.5-turbo",
  temperature=0.2,
  max_tokens=1000,  # Queremos que las respuestas sean tan largas como el modelo lo considere, sin considerar la cantidad de tokens
)

In [2]:
import tiktoken
import json

path = "../data/gse_2023_11_09.jsonl"
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')

def num_tokens_from_string(string: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
    num_tokens = len(encoding.encode(string))
    return num_tokens


len_token = []
tex_list = []
pages = []
with open(path) as f:
    for line in f:
        data = json.loads(line)
        
        text = data['text']
        pages.append(data['page'])
        len_token.append(num_tokens_from_string(text))
        tex_list.append(text)

In [3]:
import pandas as pd
df = pd.DataFrame({"pag":pages,"Num_tokens":len_token, "Texto":tex_list})
df

,pag,Num_tokens,Texto
0,1,57,Aprobada por Resolución de Presidencia de Cons...
1,2,231,Guía para el seguimiento y evaluación de polít...
2,3,76,CENTRO NACIONAL DE PLANEAMIENTO ESTRATÉGICO - ...
3,4,32,4 GUÍA PARA EL SEGUIMIENTO Y EVALUACIÓN DE POL...
4,5,22,CENTRO NACIONAL DE PLANEAMIENTO ESTRATÉGICO - ...
...,...,...,...
151,152,434,152 GUÍA PARA EL SEGUIMIENTO Y EVALUACIÓN DE P...
152,153,876,CENTRO NACIONAL DE PLANEAMIENTO ESTRATÉGICO - ...
153,154,867,154 GUÍA PARA EL SEGUIMIENTO Y EVALUACIÓN DE P...
154,155,793,CENTRO NACIONAL DE PLANEAMIENTO ESTRATÉGICO - ...


In [11]:
sum(df.Num_tokens)

90356

In [4]:
# Modelo de chat
from langchain.chat_models import ChatOpenAI
from utils import get_openai_api_key

get_openai_api_key()

chat = ChatOpenAI(
    temperature=0.0, 
    model="gpt-3.5-turbo"
    )

# Prompt template
from langchain.prompts import ChatPromptTemplate

template_string = """You are tasked with extracting a logic propositions fron the given text, remember to be caution \
and consider the proposition should be related to government planning. Consider based in informative entities, where \
informative entities are: relevant to the main topic, faithful and should present in the given Text, and located \
anywhere in the text. Finally, provides all propositions in Spanish language and in list format.

Given text: {text}
"""
prompt_template = ChatPromptTemplate.from_template(
                    template_string
                  )

message = prompt_template.format_messages(
  text = df["Texto"][151]
  )

response = chat(message)

print(response)

content='Propositions related to government planning in Spanish:\n\n1. La guía proporciona un modelo de reporte de cumplimiento de políticas nacionales y planes del SINAPLAN.\n2. El reporte de cumplimiento de políticas nacionales debe indicar el nombre de la política nacional.\n3. El reporte de cumplimiento de políticas nacionales debe indicar el nombre del ministerio rector o conductor.\n4. El reporte de cumplimiento de políticas nacionales debe indicar el problema público identificado en la política nacional.\n5. El reporte de cumplimiento de políticas nacionales debe indicar todos los objetivos prioritarios de la política nacional.\n6. El reporte de cumplimiento de políticas nacionales debe indicar el dispositivo legal que aprueba la política nacional.\n7. El resumen del reporte de cumplimiento debe describir de manera breve los principales resultados, avances, intervenciones y mejoras necesarias en la política nacional.\n8. El cumplimiento de los objetivos prioritarios de la políti

In [9]:
import re
texto = response.content

# Expresión regular para encontrar proposiciones numeradas
expresion_regular = re.compile(r'\d+\.\s(.+?)(?=\n\d+\.|$)')
# expresion_regular = re.compile(r'(\d+\.\s.+?)(?=\n\d+\.|$)')

# Buscar todas las coincidencias en el texto
propositions = expresion_regular.findall(texto)

list_propositions = [proposition for proposition in propositions]

# Imprimir la lista de proposiciones
list_propositions

['La guía proporciona un modelo de reporte de cumplimiento de políticas nacionales y planes del SINAPLAN.',
 'El reporte de cumplimiento de políticas nacionales debe indicar el nombre de la política nacional.',
 'El reporte de cumplimiento de políticas nacionales debe indicar el nombre del ministerio rector o conductor.',
 'El reporte de cumplimiento de políticas nacionales debe indicar el problema público identificado en la política nacional.',
 'El reporte de cumplimiento de políticas nacionales debe indicar todos los objetivos prioritarios de la política nacional.',
 'El reporte de cumplimiento de políticas nacionales debe indicar el dispositivo legal que aprueba la política nacional.',
 'El resumen del reporte de cumplimiento debe describir de manera breve los principales resultados, avances, intervenciones y mejoras necesarias en la política nacional.',
 'El cumplimiento de los objetivos prioritarios de la política nacional debe ser descrito en el reporte.',
 'La situación de las 

In [ ]:
def get_proposition(Texto):
  message = prompt_template.format_messages(
    text = Texto
    )
  return(message.content)

df = df.assign(
  proposition = df.apply(lambda x: get_proposition(x.Texto), axis=1)
)